In this notebook, I explore the idea of clustering by label (i.e. iterate over each label and create homogeneous clusters.) This yields overlapping clusters, hence the notebook name "fuzzy cluster" (I need to find a better term though). 

Work in progress - the clustering algorithm is implemented. Next step is to figure out how to make predictions with them. 

### imports

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from uuid import UUID

# hack because jupyter notebook doesn't work properly through my vscode for
# some reason and therefore cant import stuff from emission? remove this before
# pushing
###
import sys

sys.path.append('/Users/hlu2/Documents/GitHub/e-mission-server/')
###

import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.trip_queries as esdtq
import emission.core.get_database as edb
import mapping

### load data

In [ ]:
# to see the same outputs I described, put in the unique tokens for these users
email0 = "replace this"  # shankari
email1 = "replace this"  # tom
user0 = list(edb.get_uuid_db().find({"user_email": email0}))[0]['uuid']
user1 = list(edb.get_uuid_db().find({"user_email": email1}))[0]['uuid']
user2 = UUID('replace this')  # hannah

all_users = esta.TimeSeries.get_uuid_list()
user_list = np.append([user0, user1, user2],
                      np.random.choice(all_users, size=10, replace=False))
confirmed_trip_df_map = {}
labeled_trip_df_map = {}
expanded_labeled_trip_df_map = {}
expanded_all_trip_df_map = {}
for i in range(len(user_list)):
    u = user_list[i]
    print(u)
    ts = esta.TimeSeries.get_time_series(u)
    ct_df = ts.get_data_df("analysis/confirmed_trip")

    confirmed_trip_df_map[i] = ct_df
    labeled_trip_df_map[i] = esdtq.filter_labeled_trips(ct_df)
    expanded_labeled_trip_df_map[i] = esdtq.expand_userinputs(
        labeled_trip_df_map[i])
    expanded_all_trip_df_map[i] = esdtq.expand_userinputs(
        confirmed_trip_df_map[i])

### clustering by label

process: 
- iterate over each label
- - for each label, find clusters
- - - use dbscan to identify density cores 
- now plot all clusters, see if there are any overlaps 

In [ ]:
# quick test to make sure pipeline is working

fig = mapping.plot_clusters(expanded_labeled_trip_df_map[0],
                    alg='fuzzy',
                    loc_type='end',
                    cluster_unlabeled=False,
                    plot_unlabeled=True,
                    radii=[100, 150, 200, 250])
fig

Ok, cool, this is what I wanted. Now we have to figure out how to turn these fuzzy clusters into predictions. The nice thing about clustering by label is that raising the DBSCAN radius doesn't dramatically raise the max cluster diameter because we've pretty much eliminated any dendrites that can form between two clusters of different labels. The only time when we might have an undesirably large cluster is if we get a dendrite between 2 nearby clusters with the same purpose. 

In [ ]:
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[2],
                    alg='fuzzy',
                    loc_type='end',
                    cluster_unlabeled=False,
                    plot_unlabeled=True,
                    radii=[100, 150, 200, 250])
fig

In [ ]:
fig = mapping.plot_clusters(expanded_labeled_trip_df_map[1],
                    alg='fuzzy',
                    loc_type='end',
                    cluster_unlabeled=False,
                    plot_unlabeled=True,
                    radii=[100, 150, 200, 250])
fig